In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, HTML
from cilPlotlyFunctions import cilPrintHTML, cilTimeSeriesStyle, cilColorGradient, cilConfig, cilSlideTimeSeriesLayoutDict, cilSationMapLayoutDict

In [2]:
xls = pd.ExcelFile('../../data/Atmospheric_Domain/2.1SurfaceAirTemperature/Figure2.1/AnnualMeanSurfaceAirTemperature1900-2019.xlsx')
dataDF = pd.read_excel(xls, "Sheet1")
# remove first row as part of column names, and rename Anom colum
dataDF=dataDF[1:]
dataDF=dataDF.rename(columns={"1961-1990 Normal":"Anom"})
dataDF

,Year,Valentia,Birr/Gureen,Phoenix Park,Malin Head,Armagh,Tmean,Anom,9.549333333333333,Unnamed: 9,filter11,Std Dev (11 year average),Stdev 11 year temps,95%confidence,Unnamed: 14,Unnamed: 15,Unnamed: 16
1,1900.0,10.375000,9.016667,8.933333,9.203000,9.11,9.33,-0.219333,0,9.549333,NaN,NaN,NaN,NaN,NaN,1.270667,10.849333
2,1901.0,10.108333,8.858333,8.666667,9.011333,8.97,9.13,-0.419333,0,9.549333,NaN,NaN,NaN,NaN,NaN,-0.819333,8.649333
3,1902.0,10.475000,9.100000,8.833333,8.669667,8.76,9.17,-0.379333,0,9.549333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1903.0,10.291667,8.941667,8.716667,8.686333,8.91,9.12,-0.429333,0,9.549333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1904.0,10.158333,8.975000,8.891667,8.428000,8.90,9.08,-0.469333,0,9.549333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,2016.0,11.000000,9.700000,10.300000,10.000000,10.10,10.29,0.740667,0,9.549333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,2017.0,11.200000,10.000000,10.500000,10.300000,10.40,10.53,0.980667,0,9.549333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,2018.0,10.900000,9.900000,10.300000,9.800000,10.90,10.25,0.700667,0,9.549333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,2019.0,NaN,NaN,NaN,NaN,NaN,10.47,0.920667,0,9.549333,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Using excel derived movng average
# To use use pandas rolling function to calcualte the 11 year moving average
# of the temperature annomaly
# movingAvg = dataDF.Anom.rolling(window=11, center=True).mean() 

# Set the trace for annual mean, sets plot type and configurations
annualTrace = go.Scatter(x=dataDF.Year, 
                        y=dataDF.Tmean, 
                        name='Annual Mean', 
                        mode='markers',
                        text=dataDF.Anom,
                        marker=cilTimeSeriesStyle()['markerStyleSecondary'],
                        hovertemplate="%{x}<br>" +
                         "<b>Annual</b><br>" +
                        "Tmean: %{y:.2f}\u00b0C<br>" + 
                        "Anomaly: %{text:.2f}\u00b0C<extra></extra>"
                        )
# Set the trace for movingAvg, sets plot type and configurations
movingAvgTrace = go.Scatter(x=dataDF.Year, 
                           y=dataDF.filter11, 
                            text=dataDF['Std Dev (11 year average)'],
                           name='11 Year Moving Average',
                           mode='lines', # 'line' is default 
                           line_shape = 'spline',
                           line=cilTimeSeriesStyle()['lineStylePrimary'],
                           hovertemplate="%{x}<br>" +
                                "<b>Moving Average</b><br>" +
                                "Anomaly: %{y:.2f}\u00b0C<extra></extra>"
                           )

# regressionTrace = go.Scatter(x=dataDF.Year, 
#                            y=regressionDF, 
# #                             text=dataDF['Std Dev (11 year average)'],
#                            name='11 Year Moving Average',
#                            mode='lines', # 'line' is default 
#                            line_shape = 'spline',
#                            line=cilTimeSeriesStyle()['lineStylePrimary'],
#                            hovertemplate="<b>Moving Average</b><br>" +
#                                 "Anomaly: %{y:.2f}\u00b0C<extra></extra>"
#                            )

normalTrace = go.Scatter(x=[1990,2020], 
                        y=[0,0], 
                        name='1961-1990 Normal', 
                        mode='lines',
                        marker=cilTimeSeriesStyle()['markerStyleSecondary'],
                        hoverinfo='skip',
)

movingAvgDF=dataDF.loc[dataDF.filter11.notna()]
linearTrendPoly=np.polyfit(movingAvgDF["Year"],movingAvgDF["filter11"],1 )
linearTrendY = np.poly1d(linearTrendPoly)(movingAvgDF["Year"])
linearTrendTrace = go.Scatter(x=movingAvgDF["Year"], 
                        y=linearTrendY, 
                        name='Linear Trend',
                        line=dict(color='#00a4ae',
                                  dash='dash',
                                width=2),
                        hoverinfo='skip',
)

In [5]:
satChart = make_subplots(specs=[[{"secondary_y": True}]])

# satChart.add_trace(normalTrace,
#                secondary_y=False)
satChart.add_trace(annualTrace,
               secondary_y=True,)
satChart.add_trace(movingAvgTrace,
               secondary_y=False)
satChart.add_trace(linearTrendTrace,
               secondary_y=False)
satChart.update_layout(
                   title="<b>Mean Surface Air Temperature (1900-2019)</b>",
                   title_x=0.5, # Centers the title
                   height=450,
                   margin={"t":40, "b":0,"r":0,"l":0,},  
                   plot_bgcolor='#f7fbfd',
                   paper_bgcolor='rgba(0,0,0,0)',
                   font = dict(
                       family = "Arial",
                       size = 13,
                       color = "#7f7f7f"
                    ),
                  hovermode="x",
                    legend=dict(
                           orientation="h",
                          x=0.1,
                        y=1.06,
                          bgcolor='rgba(0,0,0,0)',
                          itemclick=False,
                          itemdoubleclick=False,
#                        legend=dict(
#                            title='<b>         Legend</b>',
#                            orientation="v",
#                           x=0.01,
#                           bgcolor='rgba(0,0,0,0)',
#                           itemclick=False,
#                           itemdoubleclick=False,
                                ), )
# Update y-axes layout seperatly due to the double y-axis chart
satChart.update_yaxes(title_text="Difference (\u00b0C) from 1961-1990 Normal",
                  secondary_y=False,
                  range=[-0.9,1.3],
                  showgrid=False,
                  dtick=0.5, # dtick sets the distance between ticks
                  tick0=0, # tick0 sets a point to map the other ticks 
                  fixedrange = True,
                  showspikes = True,
                  zeroline = True, #add a zero line 
                  zerolinecolor = '#E1AF00', #colour of zero line 
#                   linewidth = 2,
#                   spikethickness = 2,
#                   linecolor = '#356b6a'
                     )

satChart.update_yaxes(title_text="Mean Annual Temperature (\u00b0C)",
                  secondary_y=True,
                  range=[8.65,10.85],
                  showgrid=False,
                  dtick=0.5, # dtick sets the distance between ticks
                  tick0=9.55, # tick0 sets a point to map the other ticks 
                  fixedrange = True,
#                      linewidth = 2,
#                   spikethickness = 2,
#                   linecolor = '#356b6a'
                     )
                          
# X AXIS
satChart.update_xaxes(
                 title='Year',
                 fixedrange=True, #stops the users being able to zoom
                 tickformat = "000", #number format
                 showspikes = True, #show the spike lines on hover
                 spikethickness=2, #spike line thickness
#                  linewidth = 2, #width of axis line
#                  linecolor = '#356b6a'
) # colour of axis line

satChart.add_annotation(x=2015, 
                        y=0.055,
                        text="1961-1990 Normal",
                        showarrow=False,
                        font=dict(
                             color='#E1AF00'),)
config=cilConfig()
satChart.show(config=config)
# Note on chart.
# Without including the Anomaly trace the '1961-1990 Normal' does not sit on the primary y-axis 0.

In [6]:
cilPrintHTML(
    chart = satChart,
    chartName='2_1_surfaceAirTempChart',
    # sourceList is a list of dictionarys
    sourceList=[{'title':'',
                'url':''},
 ],
    include_plotlyjs=False,
    config=config
)

In [ ]:
stationsDF = pd.read_csv('../../data/Atmospheric_Domain/2.1SurfaceAirTemperature/Map2.1/Map2.1_StationTable.txt')
stationsDF

In [ ]:
stationColor = {
    'Buoy':'yellow',
    'Synoptic':'red',
    'Rainfall':'blue',
    'Climate':'green',
}
colors = [stationColor[k] for k in stationsDF['Type'].values]

In [ ]:
def my_text(df):
    my_text=['Name: '+'{}'.format(n)+'<br>'+
         'Type: '+'{}'.format(t)+'<br>'+
         'Station No.: '+'{}'.format(sN)+'<br>'+
         'County.: '+'{}'.format(cnty)+'<br>'+
         'Open Year.: '+'{}'.format(oY)+'<br>'+
         'Height: '+'{:.2f}'.format(h)+'m<br>'+
         'Easting: '+'{}'.format(easting)+'<br>'+
         'Northing: '+'{}'.format(northing)+'<br>'
         for n, t, sN, cnty, oY, h, easting, northing in zip(list(df['name']), 
                                     list(stationsDF['Type']), 
                                     list(stationsDF['Station_Nu']), 
                                     list(stationsDF['County']),
                                     list(stationsDF['Open_Year']),
                                     list(stationsDF['Height__m_']),
                                     list(stationsDF['Easting']),
                                     list(stationsDF['Northing']),
                                                            )]
    return my_text
    

In [ ]:
# my_text=['Name: '+'{}'.format(n)+'<br>'+
#          'Type: '+'{}'.format(t)+'<br>'+
#          'Station No.: '+'{}'.format(sN)+'<br>'+
#          'County.: '+'{}'.format(cnty)+'<br>'+
#          'Open Year.: '+'{}'.format(oY)+'<br>'+
#          'Height: '+'{:.2f}'.format(h)+'m<br>'+
#          'Easting: '+'{}'.format(easting)+'<br>'+
#          'Northing: '+'{}'.format(northing)+'<br>'
#          for n, t, sN, cnty, oY, h, easting, northing in zip(list(stationsDF['name']), 
#                                      list(stationsDF['Type']), 
#                                      list(stationsDF['Station_Nu']), 
#                                      list(stationsDF['County']),
#                                      list(stationsDF['Open_Year']),
#                                      list(stationsDF['Height__m_']),
#                                      list(stationsDF['Easting']),
#                                      list(stationsDF['Northing']),
#                                                             )]



In [ ]:
# data = go.Scattermapbox(
#         lon=stationsDF.Longitude,
#         lat=stationsDF.Latitude,
#         text=my_text,
#         marker=dict(color=colors,
#                     size=8),
#             hovertemplate= "%{text}" +
#             "Lat: %{lon}\u00b0<br>" +
#             "Lon: %{lat}\u00b0<br>" +
#             "<extra></extra>",
# )

# config = {
#     'displaylogo': False
# }


In [ ]:
# surfaceTempStationMap = go.Figure(data=data,layout=cilSationMapLayoutDict())
# config={"scrollZoom": True,
#         'displayModeBar': False,
#         'displaylogo': False}
# surfaceTempStationMap.show(config=config)

In [ ]:
# cilPrintHTML(
#     chart = surfaceTempStationMap,
#     chartName='surfaceTempStationMap',
#     # sourceList is a list of dictionarys
#     sourceList=[{'title':'',
#                 'url':''},
#  ],
#     include_plotlyjs=False,
#     config=config
# )

In [ ]:
buoyDF=stationsDF.loc[stationsDF['Type'] == 'Buoy']
synopticDF=stationsDF.loc[stationsDF['Type'] == 'Synoptic']
rainfallDF=stationsDF.loc[stationsDF['Type'] == 'Rainfall']
climateDF=stationsDF.loc[stationsDF['Type'] == 'Climate']

buoyTrend = go.Scattermapbox(
        name='Buoys',
        lon=buoyDF.Longitude,
        lat=buoyDF.Latitude,
        text=my_text(buoyDF),
        marker=dict(color='Yellow',
                    size=8),
            hovertemplate= "%{text}" +
            "Lat: %{lon}\u00b0<br>" +
            "Lon: %{lat}\u00b0<br>" +
            "<extra></extra>",
)
synopticTrend = go.Scattermapbox(
        name='Synoptic',
        lon=synopticDF.Longitude,
        lat=synopticDF.Latitude,
        text=my_text(synopticDF),
        marker=dict(color='Red',
                    size=8),
            hovertemplate= "%{text}" +
            "Lat: %{lon}\u00b0<br>" +
            "Lon: %{lat}\u00b0<br>" +
            "<extra></extra>",
)
rainfallTrend = go.Scattermapbox(
        name='Rainfall',
        lon=rainfallDF.Longitude,
        lat=rainfallDF.Latitude,
        text=my_text(rainfallDF),
        marker=dict(color='Blue',
                    size=8),
            hovertemplate= "%{text}" +
            "Lat: %{lon}\u00b0<br>" +
            "Lon: %{lat}\u00b0<br>" +
            "<extra></extra>",
)
climateTrend = go.Scattermapbox(
        name='Climate',
        lon=climateDF.Longitude,
        lat=climateDF.Latitude,
        text=my_text(climateDF),
        marker=dict(color='green',
                    size=8),
            hovertemplate= "%{text}" +
            "Lat: %{lon}\u00b0<br>" +
            "Lon: %{lat}\u00b0<br>" +
            "<extra></extra>",
)

In [ ]:
surfaceTempStationMap = go.Figure(data=[buoyTrend, synopticTrend, rainfallTrend, climateTrend],layout=cilSationMapLayoutDict())
config={"scrollZoom": True,
        'displayModeBar': False,
        'displaylogo': False}
surfaceTempStationMap.update_layout(
        showlegend=True,
                      legend=dict(orientation="v", #h for horizontal, 'v' is default
                                title='<b>Station Type</b>',
                                x=0.01
                              ), 
)
surfaceTempStationMap.show(config=config)

In [ ]:
cilPrintHTML(
    chart = surfaceTempStationMap,
    chartName='surfaceTempStationMap',
    # sourceList is a list of dictionarys
    sourceList=[{'title':'',
                'url':''},
 ],
    include_plotlyjs=False,
    config=config
)